# Configure a Scenario for FloodAdapt

In this notebook we are going to setup a scenario for FloodAdapt to run. This includes a weather event, any adaptation measures and future projections.


## **Step 0:** Import dependencies

In [ ]:
from pathlib import Path
from pprint import pprint

import tomli_w
from flood_adapt.objects.measures.measures import MeasureType

from DT_flood.utils.fa_scenario_utils import (
    create_event,
    create_measure,
    create_projection,
    get_database,
)
from DT_flood.utils.plot_utils import draw_database_map, list_agg_areas
from DT_flood.utils.workflow_utils import (
    create_workflow_config,
    run_fa_scenario_workflow,
)


## **Step 1:** Set paths to folders

In [ ]:
# Path to the FloodAdapt database for which we are configuring scenarios
fa_database = Path("/home/wotromp/InterTwin/workflow_refactor/fa_database/Humber_new")

database = get_database(database_path=fa_database)

## **Step 2:** Configure the weather Event

Here we will specify everything needed to setup an event. For now we will only consider scenarios which involve just a single event.

To fully specify an event we will need the following:
- A name for referencing
- Start and end time and dates
- Forcing data for the hazard models. Either paths to data files or keys for data catalogues. See HydroMT (SFINCS,WFlow) for naming conventions of data variables
    - When using a single dataset for meteorological data this needs to contain precipitation, wind speeds, and pressure. Otherwise use individual data sources for each

WFlow requires two sets of forcings, one for the warmup run, and one for the event run. See HydroMT WFlow `setup_precip_forcing` and `setup_temp_pet_forcing` which data variables should be included (conveniently ERA5 provides them all)

In [ ]:
# Get list of already existing events
events_existing = database.get_events()
print(events_existing['name'])

In [ ]:
# Name of event
event_name = 'test_event_rucio2'

# Start and end times of the event
start_time = '2013-12-04 00:00:00'
end_time = '2013-12-10 00:00:00'


In [ ]:
event = create_event(database=database, name=event_name, start_time=start_time, end_time=end_time)

## **Step 3:** Configure projections

Here we specify, where applicable, the projected future changes relevant to our scenario. These come in two flavors: physical projections and socio-economic changes. The former contains changes on the hazard side (sea level rise, land subsidence etc.) while the latter contains changes to the impact side (population growth, economic growth etc.).\
The supported projections are:\
\
**Physical Projections:**
- Sea level rise
- Land subsidence
- Rainfall increase
- Storm frequency increase

**Socio-Economic Projections:**
- Population growth
- Economic growth
- New developments:
    - Newly developed area
    - Elevation of newly developed area
    - Population growth in newly developed area

In [ ]:
# Get list of already existing projections
proj_existing = database.get_projections()
print(proj_existing['name'])

In [ ]:
# Name of projection
projection_name = 'slr_test'

# Physical projections
sea_level_rise = 1.5
rainfall_increase = 0


# Socio-Economic projections
pop_growth = 0
economic_growth = 0

In [ ]:
projection = create_projection(database=database, name=projection_name, sea_level_rise=sea_level_rise)

## **Step 4:** Configure Measures

Here we configure all the measure we want to implement for this scenario. These come in either Impact or Hazard types, either reducing the vulnerability to a hazard, or dampen the hazard itself. In either case we need to specify in which area the measure acts by specifying a geojson file or in the case of Impact types an area name recognized by the aggregation area file. For the Impact type we also need to specify which building type to act on.\
The supported measures are:\
\
**Hazard Type:**
- Floodwall
- Pump
- Water square
- Green infrastructure
- Water storage

**Impact Type:**
- Elevate properties
- Floodproof properties
- Buyout properties

All the measures needed for the scenario are collected into a single strategy.

In [ ]:
# print some useful stuff

print("Existing measures:")
meas_existing = database.get_measures()
print(meas_existing['name'])

print("Allowed measure types:")
print([e.value for e in MeasureType])

print("Existing Aggreagation area types:")
print(list_agg_areas(database=database))


In [ ]:
m, selected_geom = draw_database_map(database, agg_area_name="AdminZonesLevel3")
m

In [ ]:
selected_geom

In [ ]:
measure_name = "test_buyout_all"
measure_type = "buyout_properties"

measure_value = 1.0
property_type = "commercial"

In [ ]:
measure = create_measure(database=database, name=measure_name, measure_type=measure_type, value=measure_value, selection=selected_geom, property_type=property_type)

In [ ]:
# Double check measure set up worked (validation of measure selection type only happens when trying to fetch it from database)
print("Existing measures:")
meas_existing = database.get_measures()
print(meas_existing['name'])

## **Step 4b:** Setup a strategy

## **Step 5:** Collect everything into a single Scenario

Now that we have configured each individual element of a scenario (event, projection, measure) we can collect everything together into a single scenario definition

In [ ]:
scenario = {
    'name': "test_rucio_scenario",
    'event': event_dict,
    'projection': proj_dict,
    'strategy': strategy,
}

In [ ]:
pprint(scenario)

In [ ]:
# Write to toml file. The _toplevel suffix indicated that this is a toml file also configuring non-FloodAdapt functionalities
scenario_fn = fa_database / "input" / "scenarios" / scenario["name"] / f"{scenario['name']}_toplevel.toml"

if not scenario_fn.parent.exists():
    scenario_fn.parent.mkdir(parents=True)

with open(scenario_fn, 'wb+') as f:
    tomli_w.dump(scenario, f)

## **Step 6:** Launch workflow

With a complete definition of the scenario we can run the workflow. First we export the cwl config file based on the scenario settings we just created

In [ ]:
from pathlib import Path
from pprint import pprint
import tomli_w
from DT_flood.utils.workflow_utils import create_workflow_config, run_fa_scenario_workflow


In [ ]:
database_fn = Path("/home/wotromp/InterTwin/workflow_refactor/fa_database/Humber_new")
scenario_name = "test_rucio_scenario"

oscar_endpoint = 'https://oscar-ukri.intertwin.fedcloud.eu'
oscar_token = ""

In [ ]:
import rucio.client


client = rucio.client.Client()
client.list_accounts()

In [ ]:
create_workflow_config(database_fn, scenario_name, oscar_endpoint=oscar_endpoint, oscar_token=oscar_token)

In [ ]:
run_fa_scenario_workflow(database_fn, scenario_name, debug=False)

In [ ]:
from DT_flood.utils.fa_scenario_utils import init_scenario
import glob

In [ ]:
db, scenario = init_scenario(database_path=database_fn, scenario_name=scenario_name)

In [ ]:
db.database.events.get(scenario["event"]["name"])

In [ ]:
db.database.projections.get("empty_projection").physical_projection.sea_level_rise.value

In [ ]:
sf_adpt = db.database.static.get_overland_sfincs_model()

In [ ]:
sf_adpt._load_scenario_objects(
    db.database.scenarios.get(scenario["name"]),
    db.database.events.get(scenario["event"]["name"])
)

In [ ]:
sf_adpt.set_timing(sf_adpt._event.time)

In [ ]:
sf_adpt._model.get_model_time()

In [ ]:
vars(db)

In [ ]:
pprint(vars(db.site))

In [ ]:
fiat_path = db.output_path/"scenarios"/"test_scenario"/"Impacts"/"fiat_model"
hazard_fn, = glob.glob(str(fiat_path/"hazard")+"/*.nc")

In [ ]:
"risk" in Path(hazard_fn).name

In [ ]:
Path(hazard_fn).parent

In [ ]:
# test github acccess

In [ ]:
import xarray as xr

In [ ]:
fn = "/home/wotromp/InterTwin/workflow_refactor/fa_database/Humber_new/input/events/test_event_rucio/meteo.nc"

In [ ]:
ds = xr.open_dataset(fn)

In [ ]:
Path(ds.encoding["source"]).parent

In [ ]:
if "latitude" in ds.coords:
    print("yes")

In [ ]:
import pandas as pd

In [ ]:
(pd.to_datetime("2013-12-04 00:00:00")-pd.DateOffset(years=1)).strftime("%Y:%m:%d %H:%M:%S")